In [1]:
# Load libraries
import pandas as pd
from sklearn.ensemble import RandomForestRegressor # Import Decision Tree Classifier
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation
import numpy as np
import gensim

wv = gensim.models.word2vec.Word2Vec.load("updated_google_corpus")

TESTING_MODE = True

# load dataset
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")
test_data = test_data.drop("index",axis=1)
data = pd.concat([train_data,test_data],sort=False)

In [2]:
len(data["country"].unique())

49

In [3]:
if TESTING_MODE:
    d = pd.get_dummies(train_data[["country","region_2","province","taster_name","variety"]])
    d["points"]= train_data["points"]
    d["id"] = train_data['id']
else:
    d = pd.get_dummies(data[["country","region_2","province","taster_name","variety"]])
    d["points"]= data["points"]
    d["id"] = data['id']

In [4]:
if TESTING_MODE:
    description_vecs = train_data["description"].apply(lambda words:sum([wv[word.lower()] for word in words.split()])*1/len(words.split()))
    for i in range(300):
        d["vec_dim_"+str(i)]=description_vecs.apply(lambda v:v[i])
else:
    description_vecs = data["description"].apply(lambda words:sum([wv[word.lower()] for word in words.split()])*1/len(words.split()))
    for i in range(300):
        d["vec_dim_"+str(i)] = description_vecs.apply(lambda v:v[i])

/usr/lib/python3/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [5]:
if TESTING_MODE:
    X_train, X_test, y_train, y_test = train_test_split(d, train_data["price"], test_size=0.3, random_state=1) # 70% training and 30% test
else:
    X_train,y_train,X_test, y_test = d[:175000], data["price"][:175000], d[175000:], data["price"][175000:]

In [6]:
X_train = X_train.drop(["id"],axis=1)

In [27]:
%%time
# Create Decision Tree classifer object
clf = RandomForestRegressor(10,max_depth=20,min_samples_split=2,n_jobs=10)
# Train Decision Tree Classifer
clf = clf.fit(X_train,y_train)

ValueError: min_samples_split must be an integer greater than 1 or a float in (0.0, 1.0]; got the integer 1

In [ ]:
#Predict the response for test dataset
#ids = X_test["id"]
#X_test = X_test.drop(["id"],axis=1)

y_pred = clf.predict(X_test)

In [23]:
print(y_test[:10],y_pred[:10])

10709      9.0
27315     28.0
129593    38.0
23186     17.0
61957     18.0
128582    22.0
83535     20.0
65190     13.0
100308    28.0
24450     16.0
Name: price, dtype: float64 [17.41440966 53.57888974 30.72874516 42.7461788  18.91864412 20.25496312
 15.98065577 29.65807393 21.63100256 16.8109755 ]


In [24]:
y_pred.shape

(52500,)

In [ ]:
result = pd.DataFrame()
result['id'] = ids
result["price"] = y_pred

In [ ]:
from math import ceil
result["price"]=result["price"].apply(ceil)

In [ ]:
result.to_csv('out_rfr_.csv',index=False)

In [25]:
print("Accuracy:",metrics.accuracy_score(y_test, np.int64(y_pred)))


Accuracy: 0.0448


In [26]:
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

RMSE: 26.74973180661442


In [ ]:
20,-  : 23.649820840960377
50,-  : 22.99395012474037  
100,- : 22.884203101828
20,20 : 24.206928409441964
20,50 : 23.654831350249047
20,100 : 24.032463084435644
20,75 : 23.79979766920799
100,75 : 22.846035679936353 -> 33.87389
10,20,10: 26.076570435580027
10,20,2: 25.116716569846435
10,20,20: 26.74973180661442

In [ ]:
y_pred[np.argmin(y_pred)]